In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Load tables into DataFrames
ecom_events = pd.read_csv("ecom_events.csv")
ecom_checkouts = pd.read_csv("ecom_checkouts.csv")
ecom_users = pd.read_csv("ecom_users.csv")
ecom_products = pd.read_csv("ecom_products.csv")
ecom_retention = pd.read_csv("ecom_retention.csv")
ecom_ab_tests = pd.read_csv("ecom_ab_tests.csv")

In [ ]:
print("\n1. Users Overview")
print("-" * 50)
print("\nFirst few rows of the user dataset:")
print(ecom_users.head())

print("\nUsers Info:")
print(ecom_users.info())

print("\nUsers Statistics:")
print(ecom_users.describe())

print("\n1. Products Overview")
print("-" * 50)
print("\nFirst few rows of the product dataset:")
print(ecom_products.head())

print("\nProducts Info:")
print(ecom_products.info())

print("\nProducts Statistics:")
print(ecom_products.describe())

print("\n1. Events Overview")
print("-" * 50)
print("\nFirst few rows of the Events dataset:")
print(ecom_events.head())

print("\nEvents Info:")
print(ecom_events.info())

print("\nEvents Statistics:")
print(ecom_events.describe())

print("\n1. Checkouts Overview")
print("-" * 50)
print("\nFirst few rows of the Checkouts dataset:")
print(ecom_checkouts.head())

print("\nCheckouts Info:")
print(ecom_checkouts.info())

print("\nCheckouts Statistics:")
print(ecom_checkouts.describe())

print("\n1. Retention Overview")
print("-" * 50)
print("\nFirst few rows of the Retention dataset:")
print(ecom_retentions.head())

print("\nRetentions Info:")
print(ecom_retentions.info())

print("\nRetentions Statistics:")
print(ecom_retentions.describe())

In [ ]:
# check null and duplicate values

def check_data_issues(df, df_name):
    print(f"Checking data issues for {df_name}...\n")
    
    # Check for null values
    null_counts = df.isnull().sum()
    print(f"Null values in {df_name}:\n{null_counts[null_counts > 0]}\n")
    
    # Check for duplicate rows
    duplicate_count = df.duplicated().sum()
    print(f"Number of duplicate rows in {df_name}: {duplicate_count}\n")
    
    # Summary
    print(f"Total rows: {df.shape[0]}, Total columns: {df.shape[1]}\n")
    print("-" * 50)

check_data_issues(ecom_users, "ecom_users")
check_data_issues(ecom_products, "ecom_products")
check_data_issues(ecom_events, "ecom_events")
check_data_issues(ecom_checkouts, "ecom_checkouts")
check_data_issues(ecom_retention, "ecom_retention")

In [ ]:
# Standardize categorical values
ecom_users['device_preference'] = ecom_users['device_preference'].str.lower().str.strip()
ecom_users['preferred_payment'] = ecom_users['preferred_payment'].str.lower().str.strip()
ecom_events['device_type'] = ecom_events['device_type'].str.lower().str.strip()
ecom_checkouts['payment_method'] = ecom_checkouts['payment_method'].str.lower().str.strip()

In [ ]:
# Convert Datetime column to proper format
# Mapping of DataFrame names to their date columns
date_columns_map = {
    'ecom_users': ['join_date'],
    'ecom_checkouts': ['purchase_time'],
    'ecom_events': ['event_time'],
    'ecom_retensions': ['last_purchase_date']
}

# Convert date columns using the mapping
for df_name, date_cols in date_columns_map.items():
    df = globals().get(df_name)
    if df is not None:
        for col in date_cols:
            if col in df.columns:
                df[col] = pd.to_datetime(df[col])


In [ ]:
# Extract time-based features
ecom_events['hour_of_day'] = ecom_events['event_time'].dt.hour
ecom_events['day_of_week'] = ecom_events['event_time'].dt.dayofweek

# Compute session duration
ecom_events['session_duration_sec'] = ecom_events.groupby('session_id')['time_spent'].transform('sum')

# Price Sensitivity Categorization
def categorize_price_sensitivity(spend):
    if spend >= 500: return 'L'
    elif spend >= 100: return 'M'
    else: return 'H'
ecom_users['price_sensitivity'] = ecom_users['user_id'].map(
    ecom_retention.set_index('user_id')['total_spend'].apply(categorize_price_sensitivity))

# Cart Abandonment Count
ecom_retention['abandoned_carts'] = ecom_events.groupby('user_id')['abandonment_stage'].count()

# Calculate completed checkouts and checkout reached per user
checkouts_per_user = ecom_checkouts.groupby('user_id')['checkout_id'].count()
checkout_reached_per_user = ecom_events.groupby('user_id')['checkout_reached'].sum()
checkout_completion_rate = checkouts_per_user / checkout_reached_per_user.replace(0, np.nan)

# Assign safely to ecom_retention
ecom_retention['checkout_completion_rate'] = ecom_retention['user_id'].map(checkout_completion_rate)


In [ ]:
# Create the funnel metrics
funnel_steps = ecom_events.groupby('session_id').agg({
    'event_type': [
        ('viewed_product', lambda x: (x == 'view').any()),
        ('added_to_cart', lambda x: (x == 'cart').any()),
        ('checkout_started', lambda x: (x == 'checkout_start').any()),
        ('shipping_selected', lambda x: (x == 'shipping_selected').any()),
        ('purchased', lambda x: ((x == 'purchase') & (ecom_events.loc[x.index, 'purchase_status'] == 'success')).any())
    ]
}).droplevel(0, axis=1)  # Flatten multi-level columns

# Calculate funnel metrics
funnel_metrics = pd.DataFrame({
    'total_sessions': [len(funnel_steps)],
    'product_views': [funnel_steps['viewed_product'].sum()],
    'cart_adds': [funnel_steps['added_to_cart'].sum()],
    'checkouts_started': [funnel_steps['checkout_started'].sum()],
    'shipping_selections': [funnel_steps['shipping_selected'].sum()],
    'purchases': [funnel_steps['purchased'].sum()],
    
    'view_to_cart_rate': [round(100.0 * funnel_steps['added_to_cart'].sum() / funnel_steps['viewed_product'].sum(), 2)],
    'cart_to_checkout_rate': [round(100.0 * funnel_steps['checkout_started'].sum() / funnel_steps['added_to_cart'].sum(), 2)],
    'checkout_to_shipping_rate': [round(100.0 * funnel_steps['shipping_selected'].sum() / funnel_steps['checkout_started'].sum(), 2)],
    'shipping_to_purchase_rate': [round(100.0 * funnel_steps['purchased'].sum() / funnel_steps['shipping_selected'].sum(), 2)],
    'overall_conversion_rate': [round(100.0 * funnel_steps['purchased'].sum() / funnel_steps['viewed_product'].sum(), 2)]
})

# Display the results
funnel_metrics

In [ ]:
# Plot the funnel
plt.figure(figsize=(10, 6))
stages = ['Product Views', 'Cart Adds', 'Checkout Started', 'Shipping Selected', 'Purchases']
values = funnel_metrics.iloc[0, 1:6].values

sns.barplot(x=values, y=stages, palette='viridis')
plt.title('E-Commerce Conversion Funnel', fontsize=16)
plt.xlabel('Number of Sessions', fontsize=12)

# Add conversion rates
for i, (stage, value) in enumerate(zip(stages, values)):
    if i > 0:
        conv_rate = (value / values[i-1]) * 100
        plt.text(value + max(values)*0.05, i, f'{conv_rate:.1f}%', va='center')

plt.tight_layout()
plt.show()

In [ ]:
import plotly.graph_objects as go

# Funnel data
stages = ['Product Views', 'Cart Adds', 'Checkout Started', 'Shipping Selected', 'Purchases']
values = funnel_metrics.iloc[0, 1:6].values

# Calculate conversion rates
conversion_from_previous = [None] + [round((curr / prev) * 100, 1) for prev, curr in zip(values[:-1], values[1:])]
conversion_from_start = [round((val / values[0]) * 100, 1) for val in values]

# Build annotations for clarity
annotations = []
for i, (stage, val, conv_prev, conv_init) in enumerate(zip(stages, values, conversion_from_previous, conversion_from_start)):
    text = f"{val:,} sessions"
    if conv_prev is not None:
        text += f"<br>{conv_prev}% from previous"
    text += f"<br>{conv_init}% from start"
    annotations.append(dict(
        x=val,
        y=stage,
        text=text,
        showarrow=False,
        xanchor="left",
        yanchor="middle",
        font=dict(size=12),
        bgcolor="rgba(255,255,255,0.6)",
        bordercolor="gray",
        borderwidth=1
    ))

# Plot the funnel
fig = go.Figure(go.Funnel(
    y=stages,
    x=values,
    textinfo="value+percent previous",
    opacity=0.9,
    marker=dict(color='mediumseagreen', line=dict(width=2, color='gray'))
))

# Update layout with annotations
fig.update_layout(
    title="🛒 E-Commerce Conversion Funnel",
    annotations=annotations,
    margin=dict(l=80, r=80, t=80, b=40),
    font=dict(family="Arial", size=14),
    plot_bgcolor='rgba(0,0,0,0)'
)

fig.show()



## Cart Abandonment Analysis

In [ ]:
def cart_abandonment_analysis():
    # Calculate cart abandonment rate
    cart_sessions = ecom_events[ecom_events['event_type'] == 'cart']['session_id'].nunique()
    purchase_sessions = ecom_events[(ecom_events['event_type'] == 'purchase') & 
                             (ecom_events['purchase_status'] == 'success')]['session_id'].nunique()
    abandonment_rate = round((cart_sessions - purchase_sessions) / cart_sessions * 100, 1)
    
    # Reasons for abandonment
    abandonment_reasons = ecom_events[ecom_events['event_type'].isin(['cart_abandoned', 'checkout_abandoned'])]
    reasons_count = abandonment_reasons['abandonment_stage'].value_counts().reset_index()
    reasons_count.columns = ['abandonment_stage', 'count']
    
    # Average time spent before abandonment
    abandoned_sessions = abandonment_reasons['session_id'].unique()
    abandoned_time = ecom_events[(ecom_events['session_id'].isin(abandoned_sessions)) & 
                          (ecom_events['event_type'] == 'session_end')]
    avg_time_before_abandonment = round(abandoned_time['session_duration_sec'].mean() / 60, 1)
    
    # Visualization
    plt.figure(figsize=(12, 5))
    
    plt.subplot(1, 2, 1)
    plt.pie(reasons_count['count'], labels=reasons_count['abandonment_stage'], autopct='%1.1f%%')
    plt.title('Abandonment Reasons Distribution')
    
    plt.subplot(1, 2, 2)
    sns.barplot(x=['Cart Abandonment Rate'], y=[abandonment_rate])
    plt.title('Overall Cart Abandonment Rate')
    plt.ylabel('Percentage (%)')
    
    plt.tight_layout()
    plt.show()
    
    print(f"\nAverage time before abandonment: {avg_time_before_abandonment} minutes")
    return abandonment_rate, reasons_count

abandonment_rate, reasons_count = cart_abandonment_analysis()
print("\nCart Abandonment Analysis Results:")
print(f"Overall abandonment rate: {abandonment_rate}%")
print("Abandonment reasons:")
print(reasons_count)

## Payment Failure Analysis

In [ ]:
# Payment failure analysis
payment_events = ecom_events[
    (ecom_events['event_type'] == 'purchase') &
    (ecom_events['purchase_status'].isin(['success', 'failed']))
]

payment_stats = payment_events.groupby(['purchase_status', 'payment_method']).agg(
    count=('session_id', 'nunique'),
    avg_page_load=('page_load_sec', 'mean'),
    avg_time_spent=('time_spent', 'mean')
).reset_index()

print("\nPayment Success/Failure Rates:")
print(payment_stats.pivot(index='payment_method', columns='purchase_status', values='count'))

# Time of day impact
payment_events['hour'] = payment_events['event_time'].dt.hour
time_impact = payment_events.groupby(['hour', 'purchase_status'])['session_id'].nunique().unstack()
time_impact['failure_rate'] = time_impact['failed'] / (time_impact['success'] + time_impact['failed'])
print("\nFailure Rate by Hour:")
print(time_impact[['failure_rate']])

## Device Segment Analysis 

In [ ]:
def device_segment_analysis():
    # Merge device info from users to events
    device_events = ecom_events.merge(ecom_users[['user_id', 'device_preference']], on='user_id')
    
    # Calculate conversion rates by device
    device_funnel = device_events.groupby('device_preference').agg({
        'session_id': 'nunique',
        'event_type': lambda x: (x == 'purchase').sum()
    }).rename(columns={'session_id': 'total_sessions', 'event_type': 'purchases'})
    
    device_funnel['conversion_rate'] = round(device_funnel['purchases'] / device_funnel['total_sessions'] * 100, 1)
    
    # Calculate average session duration by device
    session_duration = device_events[device_events['event_type'] == 'session_end']
    avg_duration = session_duration.groupby('device_preference')['session_duration_sec'].mean().reset_index()
    avg_duration['session_duration_min'] = round(avg_duration['session_duration_sec'] / 60, 1)
    
    # Visualization
    plt.figure(figsize=(12, 5))
    
    plt.subplot(1, 2, 1)
    sns.barplot(x='device_preference', y='conversion_rate', data=device_funnel.reset_index())
    plt.title('Conversion Rate by Device Type')
    plt.ylabel('Conversion Rate (%)')
    
    plt.subplot(1, 2, 2)
    sns.barplot(x='device_preference', y='session_duration_min', data=avg_duration)
    plt.title('Average Session Duration by Device')
    plt.ylabel('Minutes')
    
    plt.tight_layout()
    plt.show()
    
    return device_funnel, avg_duration

device_results, duration_results = device_segment_analysis()
print("\nDevice Segment Analysis Results:")
print(device_results)
print("\nAverage Session Duration by Device:")
print(duration_results)

## Payment Method Analysis

In [ ]:
def payment_method_analysis():
    # Payment method distribution
    payment_dist = ecom_checkouts['payment_method'].value_counts(normalize=True).reset_index()
    payment_dist.columns = ['payment_method', 'percentage']
    payment_dist['percentage'] = round(payment_dist['percentage'] * 100, 1)
    
    # Payment method vs conversion
    payment_success = ecom_events[ecom_events['event_type'] == 'purchase']
    payment_success = payment_success.merge(ecom_users[['user_id', 'preferred_payment']], on='user_id')
    payment_success_rate = payment_success.groupby('preferred_payment')['purchase_status'].apply(
        lambda x: (x == 'success').mean() * 100).reset_index()
    payment_success_rate.columns = ['payment_method', 'success_rate']
    
    # Average order value by payment method
    aov_by_payment = ecom_checkouts.groupby('payment_method')['total_paid'].mean().reset_index()
    aov_by_payment['total_paid'] = round(aov_by_payment['total_paid'], 2)
    
    # Visualization
    plt.figure(figsize=(15, 5))
    
    plt.subplot(1, 3, 1)
    sns.barplot(x='payment_method', y='percentage', data=payment_dist)
    plt.title('Payment Method Distribution')
    plt.ylabel('Percentage (%)')
    
    plt.subplot(1, 3, 2)
    sns.barplot(x='payment_method', y='success_rate', data=payment_success_rate)
    plt.title('Payment Success Rate by Method')
    plt.ylabel('Success Rate (%)')
    
    plt.subplot(1, 3, 3)
    sns.barplot(x='payment_method', y='total_paid', data=aov_by_payment)
    plt.title('Average Order Value by Payment Method')
    plt.ylabel('Amount ($)')
    
    plt.tight_layout()
    plt.show()
    
    return payment_dist, payment_success_rate, aov_by_payment

payment_dist, success_rates, aov_payment = payment_method_analysis()
print("\nPayment Method Analysis Results:")
print("Payment method distribution:")
print(payment_dist)
print("\nPayment success rates:")
print(success_rates)
print("\nAverage order value by payment method:")
print(aov_payment)

## Retention Analysis

In [ ]:
def retention_analysis():
    # Cohort analysis
    ecom_users['join_month'] = ecom_users['join_date'].dt.to_period('M')
    ecom_checkouts['purchase_month'] = ecom_checkouts['purchase_time'].dt.to_period('M')
    
    cohort_data = ecom_checkouts.merge(ecom_users[['user_id', 'join_month']], on='user_id')
    cohort_data['cohort_index'] = (cohort_data['purchase_month'] - cohort_data['join_month']).apply(lambda x: x.n)
    
    cohort_pivot = cohort_data.pivot_table(index='join_month', 
                                         columns='cohort_index', 
                                         values='user_id', 
                                         aggfunc=pd.Series.nunique)
    
    cohort_size = cohort_pivot.iloc[:, 0]
    retention_matrix = cohort_pivot.divide(cohort_size, axis=0)
    
    # Calculate overall retention metrics
    repeat_customers = ecom_retention[ecom_retention['total_purchases'] > 1].shape[0]
    repeat_rate = round(repeat_customers / len(ecom_retention) * 100, 1)
    avg_purchases = round(ecom_retention['total_purchases'].mean(), 1)
    
    # Visualization
    plt.figure(figsize=(15, 5))
    
    plt.subplot(1, 2, 1)
    sns.heatmap(retention_matrix, annot=True, fmt='.0%', cmap='Blues')
    plt.title('Monthly Cohort Retention Rates')
    plt.ylabel('Cohort Month')
    plt.xlabel('Months Since First Purchase')
    
    plt.subplot(1, 2, 2)
    sns.histplot(ecom_retention['days_since_last_purchase'], bins=30, kde=True)
    plt.title('Days Since Last Purchase Distribution')
    plt.xlabel('Days')
    
    plt.tight_layout()
    plt.show()
    
    print(f"\nRepeat customer rate: {repeat_rate}%")
    print(f"Average purchases per customer: {avg_purchases}")
    return retention_matrix

retention_matrix = retention_analysis()
print("\nRetention Matrix:")
print(retention_matrix)

## Checkout Optimization Analysis

In [ ]:
def checkout_optimization_analysis():
    # Get checkout timing data
    checkout_timing = ecom_events[ecom_events['event_type'].isin(['checkout_start', 'shipping_selected', 
                                                     'payment_entered', 'purchase'])]
    
    # Calculate step completion rates
    step_completion = checkout_timing.groupby('checkout_step')['session_id'].nunique().reset_index()
    step_completion['completion_rate'] = round(step_completion['session_id'] / step_completion['session_id'].max() * 100, 1)
    
    # Calculate average time per step
    step_times = checkout_timing.groupby('checkout_step')['step_time_sec'].mean().reset_index()
    step_times['step_time_sec'] = round(step_times['step_time_sec'], 1)
    
    # Impact of page load time on conversion
    checkout_sessions = ecom_events[ecom_events['event_type'] == 'checkout_start']['session_id'].unique()
    checkout_events = ecom_events[ecom_events['session_id'].isin(checkout_sessions)]
    
    page_load_impact = checkout_events.groupby('session_id').agg({
        'page_load_sec': 'mean',
        'event_type': lambda x: 'purchase' in x.values
    }).reset_index()
    page_load_impact.columns = ['session_id', 'avg_page_load', 'converted']
    
    # Visualization
    plt.figure(figsize=(15, 5))
    
    plt.subplot(1, 3, 1)
    sns.barplot(x='checkout_step', y='completion_rate', data=step_completion)
    plt.title('Checkout Step Completion Rates')
    plt.ylabel('Completion Rate (%)')
    
    plt.subplot(1, 3, 2)
    sns.barplot(x='checkout_step', y='step_time_sec', data=step_times)
    plt.title('Average Time per Checkout Step')
    plt.ylabel('Seconds')
    
    plt.subplot(1, 3, 3)
    sns.boxplot(x='converted', y='avg_page_load', data=page_load_impact)
    plt.title('Page Load Time vs Conversion')
    plt.xlabel('Converted')
    plt.ylabel('Average Page Load (sec)')
    
    plt.tight_layout()
    plt.show()
    
    return step_completion, step_times, page_load_impact

step_completion, step_times, page_load_impact = checkout_optimization_analysis()
print("\nCheckout Optimization Analysis Results:")
print("Step completion rates:")
print(step_completion)
print("\nAverage time per step:")
print(step_times)
print("\nPage load impact on conversion:")
print(page_load_impact.groupby('converted')['avg_page_load'].describe())

## RFM (Recency, Frequency, Monetary) Analysis

In [ ]:
from datetime import datetime

def rfm_analysis():
    # Prepare RFM data
    rfm_data = ecom_checkouts.groupby('user_id').agg({
        'purchase_time': lambda x: (datetime.now() - x.max()).days,
        'checkout_id': 'count',
        'total_paid': 'sum'
    }).reset_index()
    
    rfm_data.columns = ['user_id', 'recency', 'frequency', 'monetary']
    
    # Create RFM scores with duplicate handling
    try:
        rfm_data['r_score'] = pd.qcut(rfm_data['recency'], q=4, labels=[4, 3, 2, 1], duplicates='drop')
        rfm_data['f_score'] = pd.qcut(rfm_data['frequency'].rank(method='first'), q=4, labels=[1, 2, 3, 4])
        rfm_data['m_score'] = pd.qcut(rfm_data['monetary'], q=4, labels=[1, 2, 3, 4], duplicates='drop')
    except ValueError as e:
        print(f"Error in bucketing: {e}")
        print("\nValue counts that caused problems:")
        print("Recency:", rfm_data['recency'].value_counts().head())
        print("Frequency:", rfm_data['frequency'].value_counts().head())
        print("Monetary:", rfm_data['monetary'].value_counts().head())
        raise
    
    rfm_data['rfm_score'] = rfm_data['r_score'].astype(str) + rfm_data['f_score'].astype(str) + rfm_data['m_score'].astype(str)
    
    # Segmentation (unchanged)
    segment_map = {
        r'[4][4][4]': 'Champions',
        r'[3-4][3-4][3-4]': 'Loyal Customers',
        r'[3-4][1-3][1-3]': 'Potential Loyalists',
        r'[4][1][1]': 'New Customers',
        r'[2-3][2-3][2-3]': 'Need Attention',
        r'[1-2][3-4][3-4]': 'At Risk',
        r'[1-2][1-2][1-2]': 'Hibernating',
        r'[1][1-4][1-4]': 'Lost'
    }
    
    rfm_data['segment'] = rfm_data['rfm_score'].replace(segment_map, regex=True)
    
    # Visualization (unchanged)
    plt.figure(figsize=(12, 6))
    segment_counts = rfm_data['segment'].value_counts().reset_index()
    sns.barplot(x='index', y='segment', data=segment_counts)
    plt.title('Customer Segments Distribution')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()
    
    # Statistics (unchanged)
    segment_stats = rfm_data.groupby('segment').agg({
        'recency': 'mean',
        'frequency': 'mean',
        'monetary': 'mean'
    }).reset_index()
    
    print("\nRFM Segment Statistics:")
    print(segment_stats)
    
    return rfm_data

rfm_results = rfm_analysis()
print("\nRFM Analysis Results:")
print(rfm_results.head())

## Discount Impact Analysis

In [ ]:
def discount_impact_analysis():
    # Get discount data from events
    discount_data = ecom_events[ecom_events['event_type'] == 'cart'].merge(
        ecom_products[['product_id', 'discount_eligible', 'discount_tier']], on='product_id')
    
    # Calculate conversion with/without discount
    discount_conversion = discount_data.groupby('discount_applied').agg({
        'session_id': 'nunique',
        'event_type': lambda x: (ecom_events[(ecom_events['session_id'].isin(x)) & 
                                      (ecom_events['event_type'] == 'purchase') & 
                                      (ecom_events['purchase_status'] == 'success')]['session_id'].nunique())
    }).reset_index()
    
    discount_conversion.columns = ['discount_applied', 'sessions', 'purchases']
    discount_conversion['conversion_rate'] = round(discount_conversion['purchases'] / discount_conversion['sessions'] * 100, 1)
    
    # Discount tier analysis
    discount_tier_data = discount_data[discount_data['discount_applied'] == 'TRUE']
    tier_conversion = discount_tier_data.groupby('discount_tier').agg({
        'session_id': 'nunique',
        'event_type': lambda x: (ecom_events[(ecom_events['session_id'].isin(x)) & 
                                      (ecom_events['event_type'] == 'purchase') & 
                                      (ecom_events['purchase_status'] == 'success')]['session_id'].nunique())
    }).reset_index()
    
    tier_conversion.columns = ['discount_tier', 'sessions', 'purchases']
    tier_conversion['conversion_rate'] = round(tier_conversion['purchases'] / tier_conversion['sessions'] * 100, 1)
    
    # Visualization
    plt.figure(figsize=(15, 5))
    
    plt.subplot(1, 2, 1)
    sns.barplot(x='discount_applied', y='conversion_rate', data=discount_conversion)
    plt.title('Conversion Rate: Discount vs No Discount')
    plt.ylabel('Conversion Rate (%)')
    
    plt.subplot(1, 2, 2)
    sns.barplot(x='discount_tier', y='conversion_rate', data=tier_conversion)
    plt.title('Conversion Rate by Discount Tier')
    plt.ylabel('Conversion Rate (%)')
    
    plt.tight_layout()
    plt.show()
    
    return discount_conversion, tier_conversion

discount_results, tier_results = discount_impact_analysis()
print("\nDiscount Impact Analysis Results:")
print("Discount vs no discount:")
print(discount_results)
print("\nBy discount tier:")
print(tier_results)

## Shipping Fee Analysis

In [ ]:
def shipping_fee_analysis():
    # Get shipping fee data
    shipping_data = ecom_events[ecom_events['event_type'] == 'shipping_selected']
    
    # Shipping fee vs conversion
    shipping_conversion = shipping_data.groupby('shipping_fee').agg({
        'session_id': 'nunique',
        'event_type': lambda x: (ecom_events[(ecom_events['session_id'].isin(x)) & 
                                      (ecom_events['event_type'] == 'purchase') & 
                                      (ecom_events['purchase_status'] == 'success')]['session_id'].nunique())
    }).reset_index()
    
    shipping_conversion.columns = ['shipping_fee', 'sessions', 'purchases']
    shipping_conversion['conversion_rate'] = round(shipping_conversion['purchases'] / shipping_conversion['sessions'] * 100, 1)
    
    # Free shipping impact
    shipping_conversion['free_shipping'] = shipping_conversion['shipping_fee'] == 0
    free_shipping_impact = shipping_conversion.groupby('free_shipping')['conversion_rate'].mean().reset_index()
    
    # A/B test results
    ecom_ab_results = ecom_ab_tests.merge(ecom_events[ecom_events['event_type'] == 'purchase'][['session_id', 'purchase_status']], 
                              on='session_id', how='left')
    ecom_ab_results['converted'] = ecom_ab_results['purchase_status'] == 'success'
    
    variant_conversion = ecom_ab_results.groupby('variant')['converted'].mean().reset_index()
    variant_conversion['conversion_rate'] = round(variant_conversion['converted'] * 100, 1)
    
    # Visualization
    plt.figure(figsize=(15, 5))
    
    plt.subplot(1, 2, 1)
    sns.barplot(x='free_shipping', y='conversion_rate', data=free_shipping_impact)
    plt.title('Conversion Rate: Free Shipping vs Paid Shipping')
    plt.ylabel('Conversion Rate (%)')
    
    plt.subplot(1, 2, 2)
    sns.barplot(x='variant', y='conversion_rate', data=variant_conversion)
    plt.title('A/B Test: Shipping Threshold Variants')
    plt.ylabel('Conversion Rate (%)')
    
    plt.tight_layout()
    plt.show()
    
    return free_shipping_impact, variant_conversion

free_shipping_results, ab_test_results = shipping_fee_analysis()
print("\nShipping Fee Impact Analysis Results:")
print("Free shipping impact:")
print(free_shipping_results)
print("\nA/B test results:")
print(ab_test_results)

## Time Based Analysis

In [ ]:
def time_based_analysis():
    # Hourly patterns
    ecom_events['hour'] = pd.to_datetime(ecom_events['event_time']).dt.hour
    hourly_activity = ecom_events.groupby('hour')['session_id'].nunique().reset_index()
    
    # Day of week patterns
    ecom_events['day_of_week'] = pd.to_datetime(ecom_events['event_time']).dt.day_name()
    dow_activity = ecom_events.groupby('day_of_week')['session_id'].nunique().reset_index()
    
    # Monthly trends
    ecom_events['month'] = pd.to_datetime(ecom_events['event_time']).dt.to_period('M')
    monthly_trends = ecom_events.groupby('month')['session_id'].nunique().reset_index()
    monthly_trends['month'] = monthly_trends['month'].astype(str)
    
    # Conversion rate by time
    purchase_hour = ecom_events[ecom_events['event_type'] == 'purchase']
    purchase_hour = purchase_hour[purchase_hour['purchase_status'] == 'success']
    purchase_hour['hour'] = pd.to_datetime(purchase_hour['event_time']).dt.hour
    hourly_conversion = purchase_hour.groupby('hour')['session_id'].nunique().reset_index()
    hourly_conversion = hourly_conversion.merge(hourly_activity, on='hour')
    hourly_conversion['conversion_rate'] = round(hourly_conversion['session_id_x'] / hourly_conversion['session_id_y'] * 100, 1)
    
    # Visualization
    plt.figure(figsize=(15, 10))
    
    plt.subplot(2, 2, 1)
    sns.lineplot(x='hour', y='session_id', data=hourly_activity)
    plt.title('Hourly Activity Pattern')
    plt.ylabel('Number of Sessions')
    
    plt.subplot(2, 2, 2)
    sns.barplot(x='day_of_week', y='session_id', data=dow_activity, 
               order=['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'])
    plt.title('Activity by Day of Week')
    plt.ylabel('Number of Sessions')
    plt.xticks(rotation=45)
    
    plt.subplot(2, 2, 3)
    sns.lineplot(x='month', y='session_id', data=monthly_trends)
    plt.title('Monthly Activity Trend')
    plt.ylabel('Number of Sessions')
    plt.xticks(rotation=45)
    
    plt.subplot(2, 2, 4)
    sns.lineplot(x='hour', y='conversion_rate', data=hourly_conversion)
    plt.title('Hourly Conversion Rate')
    plt.ylabel('Conversion Rate (%)')
    
    plt.tight_layout()
    plt.show()
    
    return hourly_activity, dow_activity, monthly_trends, hourly_conversion

hourly_results, dow_results, monthly_results, hourly_conversion = time_based_analysis()
print("\nTime-Based Analysis Results:")
print("Hourly activity:")
print(hourly_results)
print("\nDay of week activity:")
print(dow_results)
print("\nMonthly trends:")
print(monthly_results)
print("\nHourly conversion rates:")
print(hourly_conversion)

## Product Category Analysis

In [ ]:
def product_category_analysis():
    # Merge product data with events
    category_data = ecom_events.merge(ecom_products[['product_id', 'category_code', 'brand']], on='product_id')
    
    # Category popularity
    category_popularity = category_data.groupby('category_code')['session_id'].nunique().reset_index()
    category_popularity.columns = ['category', 'sessions']
    category_popularity['percentage'] = round(category_popularity['sessions'] / category_popularity['sessions'].sum() * 100, 1)
    
    # Category conversion rates
    category_conversion = category_data[category_data['event_type'] == 'view'].groupby('category_code')['session_id'].nunique().reset_index()
    category_conversion.columns = ['category', 'views']
    
    purchases = category_data[(category_data['event_type'] == 'purchase') & 
                            (category_data['purchase_status'] == 'success')]
    category_purchases = purchases.groupby('category_code')['session_id'].nunique().reset_index()
    category_purchases.columns = ['category', 'purchases']
    
    category_conversion = category_conversion.merge(category_purchases, on='category', how='left').fillna(0)
    category_conversion['conversion_rate'] = round(category_conversion['purchases'] / category_conversion['views'] * 100, 1)
    
    # Price difference impact
    price_diff_data =ecom_products.merge(
        category_data[category_data['event_type'] == 'purchase'][['product_id', 'session_id']],
        on='product_id'
    )
    
    price_diff_impact = price_diff_data.groupby(pd.cut(price_diff_data['price_diff_pct'], 
                                                     bins=[-100, -10, 0, 10, 100]))['session_id'].nunique().reset_index()
    price_diff_impact.columns = ['price_diff_range', 'purchases']
    
    # Visualization
    plt.figure(figsize=(15, 10))
    
    plt.subplot(2, 2, 1)
    sns.barplot(x='percentage', y='category', data=category_popularity.sort_values('percentage', ascending=False))
    plt.title('Category Popularity')
    plt.xlabel('Percentage of Sessions (%)')
    
    plt.subplot(2, 2, 2)
    sns.barplot(x='conversion_rate', y='category', 
               data=category_conversion.sort_values('conversion_rate', ascending=False))
    plt.title('Category Conversion Rates')
    plt.xlabel('Conversion Rate (%)')
    
    plt.subplot(2, 2, 3)
    sns.barplot(x='price_diff_range', y='purchases', data=price_diff_impact)
    plt.title('Price Difference Impact on Purchases')
    plt.xlabel('Price Difference Range (%)')
    plt.ylabel('Number of Purchases')
    plt.xticks(rotation=45)
    
    plt.tight_layout()
    plt.show()
    
    return category_popularity, category_conversion, price_diff_impact

category_popularity, category_conversion, price_diff_impact = product_category_analysis()
print("\nProduct Category Analysis Results:")
print("Category popularity:")
print(category_popularity)
print("\nCategory conversion rates:")
print(category_conversion)
print("\nPrice difference impact:")
print(price_diff_impact)